# **Pipeline ETL et Modélisation : Structuration d’un Data Warehouse Retail**

## **Présentation du projet**

Ce business case vise à construire un pipeline de données complet à partir d’un jeu de données brut. L’objectif est de transformer les données brutes en un schéma adapté pour un **Data Warehouse**, permettant d’effectuer des analyses efficaces. Nous débuterons par une exploration du schéma initial, qui nécessitera une **dénormalisation** afin de répondre aux besoins analytiques spécifiques.

---

### **Contexte**
Le jeu de données utilisé dans ce projet provient d'une entreprise de vente au détail souhaitant optimiser ses opérations commerciales. Ce jeu de données contient des informations détaillées sur plusieurs aspects de l'activité, notamment :

- **Suivi des commandes clients** : Identifier qui achète quoi, quand, et dans quel magasin.
- **Gestion des stocks** : Suivre les produits commandés et gérer les niveaux d'approvisionnement.
- **Analyse des performances des magasins** : Mesurer l'efficacité des magasins en fonction des commandes traitées, des ventes réalisées, et des taux de taxe appliqués.
- **Optimisation des coûts** : Évaluer les coûts des approvisionnements, particulièrement ceux des articles périssables, pour mieux gérer les marges.

L’objectif global pour l’entreprise est de mieux comprendre son activité et de prendre des décisions éclairées concernant :
- La gestion des stocks.
- L’optimisation des prix.
- La maximisation des ventes.

---

### **Objectifs et technologies**
Le projet repose sur l’utilisation de **DBT (Data Build Tool)** pour orchestrer les transformations de données, et d’**Amazon Redshift** pour héberger l’entrepôt de données. Ce cadre technique nous permettra de travailler efficacement sur la transformation et la structuration des données tout en améliorant nos compétences en **SQL** et en modélisation.

#### **Objectifs du projet :**
1. **Créer un schéma adapté pour un Data Warehouse** :
   - Simplifier les requêtes analytiques.
   - Optimiser les performances pour l’analyse.

2. **Implémenter un pipeline de données reproductible** :
   - Transformer les données brutes en tables dénormalisées prêtes à l’analyse.

3. **Améliorer les compétences techniques** :
   - Appliquer les concepts de modélisation des données.
   - Renforcer l'expertise en **ETL (Extract, Transform, Load)** et en **data modeling**.

---

## **Etapes**

### 1. Analyse et Modélisation des données
### 2. Intégration des données
### 3. Transformation des données


---

## **1. Analyse et Modélisation des données**

### **1.1 Analyse des données brutes**

- Cette étape vise à **explorer et analyser les fichiers CSV bruts fournis, identifier les relations, créer le schéma de la structure de la base de données**, afin de préparer la modélisation des données.

#### **Import des données**

In [1]:
import pandas as pd

# Charger les fichiers CSV dans des DataFrames
df_customers = pd.read_csv("../data/datasets/raw_customers.csv")
df_items = pd.read_csv("../data/datasets/raw_items.csv")
df_orders = pd.read_csv("../data/datasets/raw_orders.csv", parse_dates=['ordered_at']) # Forcer la conversion au format date
df_products = pd.read_csv("../data/datasets/raw_products.csv")
df_stores = pd.read_csv("../data/datasets/raw_stores.csv", parse_dates=['opened_at']) # Forcer la conversion au format date
df_supplies = pd.read_csv("../data/datasets/raw_supplies.csv")

#### **Table `customers`**

In [2]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      930 non-null    object
 1   name    930 non-null    object
dtypes: object(2)
memory usage: 14.7+ KB


In [3]:
df_customers.head()

,id,name
0,ae3a050d-287f-4257-a778-cdb4206aa012,Anthony Wells
1,33a51363-1989-4967-93a1-b8e225b91e84,Scott Thompson
2,2f8f7de0-4c62-4b87-8494-6a50d68052ad,Michael Walker
3,46f7be1b-f2bc-4a91-a7c3-88c141564e31,Alicia Clark
4,c616a2a2-5efc-4e22-9db1-50dfa4d331d1,Dawn Williams


In [4]:
# Vérification des doublons pour 'id'
df_customers['id'].duplicated().sum()

np.int64(0)

#### **Table `items`**

In [5]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90183 entries, 0 to 90182
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        90183 non-null  object
 1   order_id  90183 non-null  object
 2   sku       90183 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [6]:
df_items.head()

,id,order_id,sku
0,6136835c-cfa4-4308-bd65-440f7b4cc232,363ab5f2-78b4-4227-9333-9701ef388434,BEV-001
1,1a5fc7f0-1573-454a-ae79-576673c280ed,363ab5f2-78b4-4227-9333-9701ef388434,JAF-002
2,d32d93ca-250b-47b2-b640-cb482eb59f8f,d6cdb882-aa8c-4b5a-a478-4321eff284e6,BEV-001
3,8dabb27c-fcc2-4513-8adf-886cdc5693aa,ca934a93-aca1-4aeb-9786-efb0da1ee26a,BEV-004
4,6849a62f-43ee-464c-8f1f-9d597f636e3b,ecd64383-c4df-41aa-b270-f3b110057558,BEV-002


In [7]:
# Vérification des doublons pour 'id'
df_items['id'].duplicated().sum()

np.int64(0)

#### **Table `orders`**

In [8]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63148 entries, 0 to 63147
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          63148 non-null  object        
 1   customer    63148 non-null  object        
 2   ordered_at  63148 non-null  datetime64[ns]
 3   store_id    63148 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [9]:
df_orders.head()

,id,customer,ordered_at,store_id
0,363ab5f2-78b4-4227-9333-9701ef388434,ae3a050d-287f-4257-a778-cdb4206aa012,2016-09-01 14:49:00,a9128331-08d7-41a2-b615-21283eee21cf
1,d6cdb882-aa8c-4b5a-a478-4321eff284e6,33a51363-1989-4967-93a1-b8e225b91e84,2016-09-01 08:21:00,a9128331-08d7-41a2-b615-21283eee21cf
2,ca934a93-aca1-4aeb-9786-efb0da1ee26a,2f8f7de0-4c62-4b87-8494-6a50d68052ad,2016-09-01 08:59:00,a9128331-08d7-41a2-b615-21283eee21cf
3,ecd64383-c4df-41aa-b270-f3b110057558,46f7be1b-f2bc-4a91-a7c3-88c141564e31,2016-09-01 16:42:00,a9128331-08d7-41a2-b615-21283eee21cf
4,adae1fcd-20b7-4f40-af1c-dc121d648d4f,c616a2a2-5efc-4e22-9db1-50dfa4d331d1,2016-09-01 09:07:00,a9128331-08d7-41a2-b615-21283eee21cf


In [10]:
# Vérification des doublons pour 'id'
df_orders['id'].duplicated().sum()

np.int64(0)

#### **Table `products`**

In [11]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          10 non-null     object
 1   name         10 non-null     object
 2   type         10 non-null     object
 3   price        10 non-null     int64 
 4   description  10 non-null     object
dtypes: int64(1), object(4)
memory usage: 532.0+ bytes


In [12]:
df_products.head()

,sku,name,type,price,description
0,JAF-001,nutellaphone who dis?,jaffle,1100,nutella and banana jaffle
1,JAF-002,doctor stew,jaffle,1100,house-made beef stew jaffle
2,JAF-003,the krautback,jaffle,1200,lamb and pork bratwurst with house-pickled cab...
3,JAF-004,flame impala,jaffle,1400,pulled pork and pineapple al pastor marinated ...
4,JAF-005,mel-bun,jaffle,1200,"melon and minced beef bao, in a jaffle, savory..."


In [13]:
# Vérification des doublons pour 'sku'
df_products['sku'].duplicated().sum()

np.int64(0)

#### **Table `stores`**

In [14]:
df_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         6 non-null      object        
 1   name       6 non-null      object        
 2   opened_at  6 non-null      datetime64[ns]
 3   tax_rate   6 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 324.0+ bytes


In [15]:
df_stores.head()

,id,name,opened_at,tax_rate
0,a9128331-08d7-41a2-b615-21283eee21cf,Philadelphia,2016-09-01,0.0600
1,a2a24e87-dec8-4f5d-9c9e-0e9849529489,Brooklyn,2017-03-12,0.0400
2,e1a600e7-e70a-43a8-96fd-a6dd263b50df,Chicago,2018-04-29,0.0625
3,0d16236e-9a7f-4cd6-a9ec-24b16f716316,San Francisco,2018-05-09,0.0750
4,ca0b89cc-6d98-4b87-88e3-5a0d95c6567f,New Orleans,2019-03-10,0.0400


In [16]:
# Vérification des doublons pour 'id'
df_stores['id'].duplicated().sum()

np.int64(0)

#### **Table `supplies`**

In [17]:
df_supplies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          65 non-null     object
 1   name        65 non-null     object
 2   cost        65 non-null     int64 
 3   perishable  65 non-null     bool  
 4   sku         65 non-null     object
dtypes: bool(1), int64(1), object(3)
memory usage: 2.2+ KB


In [18]:
df_supplies.head()

,id,name,cost,perishable,sku
0,SUP-001,compostable cutlery - knife,7,False,JAF-001
1,SUP-002,cutlery - fork,7,False,JAF-001
2,SUP-003,serving boat,11,False,JAF-001
3,SUP-004,napkin,4,False,JAF-001
4,SUP-009,bread,33,True,JAF-001


In [19]:
# Vérification des doublons pour 'id'
df_supplies['id'].duplicated().sum()

np.int64(36)

In [20]:
df_supplies.duplicated().sum()

np.int64(0)

In [21]:
# Vérification des lignes en doublon pour 'id'
df_supplies[df_supplies['id'].duplicated(keep=False)]

,id,name,cost,perishable,sku
0,SUP-001,compostable cutlery - knife,7,False,JAF-001
1,SUP-002,cutlery - fork,7,False,JAF-001
2,SUP-003,serving boat,11,False,JAF-001
3,SUP-004,napkin,4,False,JAF-001
4,SUP-009,bread,33,True,JAF-001
7,SUP-001,compostable cutlery - knife,7,False,JAF-002
8,SUP-002,cutlery - fork,7,False,JAF-002
9,SUP-003,serving boat,11,False,JAF-002
10,SUP-004,napkin,4,False,JAF-002
11,SUP-009,bread,33,True,JAF-002


- Les doublons observés dans `id` sont dus à une relation multiple entre les approvisionnements (`supplies`) et les produits (`sku`). Cela ne semble pas être une anomalie mais un choix de conception des données.
- Pour garantir l'unicité, il faudra certainement combiner les colonnes `id` et `sku` comme clé unique.

#### **Résumé des Observations**

Le jeu de données contient plusieurs tables interconnectées.

Voici les observations sur leur structure et les relations potentielles, basées sur la description des données et l'analyse réalisée :

---

##### **raw_customers.csv**
- **Description** : Contient des informations sur les clients.
- **Colonnes** :
  - `id` (PK) : Identifiant unique du client.
  - `name` : Nom du client.
- **Observations** :
  - La colonne `id` est une clé primaire unique, sans valeur manquante.
- **Utilité** : Cette table servira de dimension dans le schéma, représentant les clients.

---

##### **raw_items.csv**
- **Description** : Contient les détails des articles commandés.
- **Colonnes** :
  - `id` (PK) : Identifiant unique de l'article commandé.
  - `order_id` (FK) : Référence à une commande (lié à `raw_orders.id`).
  - `sku` : Code de gestion des stocks pour l'article (lié à `raw_products.sku`).
- **Observations** :
  - La colonne `id` est une clé primaire unique.
  - La colonne `order_id` agit comme une clé étrangère vers `raw_orders`.
  - La colonne `sku` est liée à `raw_products`.
- **Utilité** : Cette table pourrait être utilisée comme table de faits, reliant les commandes et les produits.

---

##### **raw_orders.csv**
- **Description** : Contient les informations sur les commandes passées.
- **Colonnes** :
  - `id` (PK) : Identifiant unique de la commande.
  - `customer_id` (FK) : Référence à un client (lié à `raw_customers.id`).
  - `ordered_at` : Date et heure de la commande.
  - `store_id` (FK) : Référence au magasin où la commande a été passée (lié à `raw_stores.id`).
- **Observations** :
  - La colonne `id` est une clé primaire unique.
  - Les colonnes `customer_id` et `store_id` agissent comme des clés étrangères vers `raw_customers` et `raw_stores`, respectivement.
- **Utilité** : Cette table servira de table de faits ou de lien avec d'autres dimensions.

---

##### **raw_products.csv**
- **Description** : Détails des produits disponibles.
- **Colonnes** :
  - `sku` (PK) : Code de gestion des stocks pour le produit.
  - `name` : Nom du produit.
  - `type` : Type ou catégorie du produit.
  - `price` : Prix du produit.
  - `description` : Description du produit.
- **Observations** :
  - La colonne `sku` est une clé primaire unique.
  - Les informations de prix et de type peuvent être utilisées pour des analyses détaillées.
- **Utilité** : Table de dimension décrivant les produits.

---

##### **raw_stores.csv**
- **Description** : Informations sur les magasins.
- **Colonnes** :
  - `id` (PK) : Identifiant unique du magasin.
  - `name` : Nom du magasin / Localisation, nom de la ville
  - `opened_at` : Date d'ouverture du magasin. (supprimer les heures)
  - `tax_rate` : Taux de taxe applicable au magasin.
- **Observations** :
  - La colonne `id` est une clé primaire unique.
  - La colonne `tax_rate` pourrait être utilisée pour calculer les taxes sur les commandes.
- **Utilité** : Table de dimension décrivant les magasins.

---

##### **raw_supplies.csv**
- **Description** : Contient des informations sur les approvisionnements.
- **Colonnes** :
  - `id` : Identifiant de l'approvisionnement (non unique).
  - `name` : Nom de l'approvisionnement.
  - `cost` : Coût de l'approvisionnement.
  - `perishable` : Indique si l'approvisionnement est périssable.
  - `sku` (FK) : Code de gestion des stocks pour l'approvisionnement (lié à `raw_products.sku`).
- **Observations** :
  - La colonne `id` contient des doublons. Chaque `id` est associé à plusieurs valeurs distinctes de `sku`. Cela suggère que la véritable unicité réside dans la combinaison des colonnes `id` et `sku`.
  - La colonne `sku` agit comme une clé étrangère vers `raw_products`.
- **Utilité** : Cette table est utile pour analyser les coûts et les approvisionnements liés aux produits. La clé primaire pourrait être redéfinie comme une combinaison de `id` et `sku`.

---

#### **Relations Identifiées**
- **Clés primaires (PK)** :
  - Chaque table possède une colonne unique utilisée comme clé primaire (`id` ou `sku`), sauf pour `raw_supplies`, où la combinaison `id` + `sku` est nécessaire pour garantir l'unicité.

- **Clés étrangères (FK)** :
  - `raw_orders.customer_id` → `raw_customers.id`
  - `raw_orders.store_id` → `raw_stores.id`
  - `raw_items.order_id` → `raw_orders.id`
  - `raw_items.sku` → `raw_products.sku`
  - `raw_supplies.sku` → `raw_products.sku`

- **Relations principales** :
  - Les commandes (`raw_orders`) relient les clients (`raw_customers`), les magasins (`raw_stores`), et les articles commandés (`raw_items`).
  - Les produits (`raw_products`) sont liés aux articles (`raw_items`) et aux approvisionnements (`raw_supplies`).

### **1.2 Schéma de la Base de Données Relationnelle Brute**

- Le schéma de la base de données brute reflète l’organisation des données opérationnelles de l'entreprise et identifie les relations entre les tables.

- Pour visualiser ce schéma, nous utilisons **dbdiagram.io**, un outil adapté pour représenter les tables, leurs clés primaires (PK) et clés étrangères (FK).

- L’observation des PK et FK permettra d’établir les relations entre les tables et de définir leurs cardinalités.

- L’objectif est de préparer un diagramme relationnel clair servant de base à la dénormalisation en vue de la création d’un Data Warehouse optimisé.

![Schéma de la BDD Brute](../data/images/Schema%20ETL%20Cloud%20Computing%20BDD%20brute.png)

#### **Résumé des Cardinalités des Relations**

En analysant le schéma de la base de données brute, les cardinalités des relations entre les tables sont les suivantes :

1. **raw_customers → raw_orders** :
   - Relation **1:N** : Un client peut passer plusieurs commandes, mais chaque commande est associée à un seul client.

2. **raw_orders → raw_items** :
   - Relation **1:N** : Une commande peut contenir plusieurs articles, mais chaque article appartient à une seule commande.

3. **raw_items → raw_products** :
   - Relation **N:1** : Plusieurs articles peuvent correspondre au même produit, mais chaque article fait référence à un seul produit.

4. **raw_orders → raw_stores** :
   - Relation **N:1** : Plusieurs commandes peuvent être associées au même magasin, mais chaque commande est liée à un seul magasin.

5. **raw_products → raw_supplies** :
   - Relation **1:N** : Un produit peut être lié à plusieurs approvisionnements, mais chaque approvisionnement est associé à un seul produit.

## **2. Intégration des données**

### **Transfert des données brutes vers AWS Redshift**

- Dans cette section, nous intégrons les données brutes sous forme de tables dans un cluster Redshift Serverless, en utilisant un bucket S3 comme stockage intermédiaire.

Les étapes principales incluent :
1. Téléversement des fichiers CSV dans un bucket S3.
2. Connexion à Redshift à l'aide de SQLAlchemy.
3. Création des tables dans Redshift en fonction des structures des fichiers CSV.
4. Transfert des données depuis S3 vers Redshift.

In [ ]:
import boto3
import json
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
from botocore.exceptions import ClientError
from dotenv import load_dotenv

# On charge les informations stockées dans le fichier ".env".
load_dotenv()

def get_secret() -> dict[str, str]:
    """
    Cette fonction sert à récupérer les informations de connexion à Redshift.
    """
    secret_name = "redshift/credentials2" # Remplacez la chaine de caractères par votre secret_name
    region_name = "eu-west-3" # Changez la région si nécessaire

    session = boto3.session.Session()
    client = session.client(service_name="secretsmanager", region_name=region_name)

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        raise e

    secret = get_secret_value_response["SecretString"]
    secret_dict = json.loads(secret)

    return secret_dict

def infer_redshift_schema(df: pd.DataFrame, table: str) -> str:
    """
    Cette fonction sert à créer une requête SQL pour créer la table à partir d'un fichier CSV.
    Les types de données de pandas et de Redshift étant différent, on créé un dictionnaire pour le mapping.
    """
    dtype_map = {
        'int64': 'BIGINT',
        'float64': 'FLOAT',
        'object': 'VARCHAR(255)',
        'bool': 'BOOLEAN',
        'datetime64[ns]': 'TIMESTAMP',
    }
    columns = []
    for col, dtype in zip(df.columns, df.dtypes):
        redshift_type = dtype_map.get(str(dtype), 'VARCHAR(255)')
        columns.append(f"{col} {redshift_type}")
    create_table_sql = f"""CREATE TABLE raw_{table} (
        {',    '.join(columns)}
        );"""
    return create_table_sql

if __name__ == "__main__":
    # On récupère les informations de connexion à Redshift
    secret = get_secret()
    username = secret.get("username")
    password = secret.get("password")
    host = secret.get("host")
    database = "dev"
    port = secret.get("port")

    engine = create_engine(
        f"redshift+psycopg2://{username}:{password}@{host}:{port}/{database}"
    )

    # Liste de toutes les tables 
    tables = ["customers", "items", "orders", "products", "stores", "supplies"]

    # Pour chaque table de la liste, on créée une table sur Redshift et on insère les données.
    for table in tables:
        print(f"Table : {table}")
        df = pd.read_csv(f"../data/datasets/raw_{table}.csv")

        with engine.connect() as conn:
            create_table_sql = infer_redshift_schema(df, table)
            conn.execute(text(create_table_sql))
            try:
                access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
                secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
                bucket_name = "bucket-etl-cloud" # Changez le nom du bucket par le nom que vous avez choisi
                copy_query = f"""
                    COPY raw_{table}
                    FROM 's3://{bucket_name}/raw_{table}.csv'
                    CREDENTIALS 'aws_access_key_id={access_key_id};aws_secret_access_key={secret_access_key}'
                    CSV IGNOREHEADER 1;
                    """
                conn.execute(text(copy_query).execution_options(autocommit=True))
                print(f"Table {table} created")
                print("-"*10)
            except SQLAlchemyError as e:
                print(e)

Table : customers
Table customers created
----------
Table : items
Table items created
----------
Table : orders
Table orders created
----------
Table : products
Table products created
----------
Table : stores
Table stores created
----------
Table : supplies
Table supplies created
----------


![Redshift Table Customers](../data/images/redshift_customers_brut.png)

![Redshift Table Customers](../data/images/redshift_customers_brut.png)

![Redshift Table Customers](../data/images/redshift_customers_brut.png)

![Redshift Table Customers](../data/images/redshift_customers_brut.png)

![Redshift Table Customers](../data/images/redshift_customers_brut.png)

![Redshift Table Customers](../data/images/redshift_customers_brut.png)

### **Conclusion : Intégration des données dans Redshift**

Pour l'intégration des données dans Redshift, nous avons suivi un processus structuré en plusieurs étapes, en utilisant des outils et modules spécifiques :

1. **Modules utilisés** :
   - **`boto3`** : Pour accéder aux services AWS, y compris Secrets Manager pour récupérer les informations d'identification.
   - **`pandas`** : Pour lire les fichiers CSV et analyser leur structure.
   - **`sqlalchemy`** : Pour établir une connexion à Redshift et exécuter les requêtes SQL.
   - **`dotenv`** : Pour charger les variables d'environnement sensibles (clés AWS, identifiants Redshift).

2. **Étapes principales** :
   - Récupération des informations d'identification depuis **AWS Secrets Manager**.
   - Création automatique des requêtes SQL pour définir la structure des tables Redshift, grâce à une fonction de mapping (`infer_redshift_schema`).
   - Téléversement des fichiers CSV sur un bucket S3, suivi de leur importation dans Redshift via la commande SQL **`COPY`**.

3. **Résultat final** :
   - Les 6 tables suivantes ont été créées et peuplées dans Redshift :
     - **`raw2_customers`**
     - **`raw2_items`**
     - **`raw2_orders`**
     - **`raw2_products`**
     - **`raw2_stores`**
     - **`raw2_supplies`**

Grâce à cette intégration, les données brutes sont maintenant disponibles dans Redshift, prêtes pour des transformations et analyses ultérieures.

## **3. Transformation des Données**

- Cette partie du projet a pour objectif de transformer un jeu de données brut en une base de données analytique dénormalisée.

- Nous utiliserons DBT (Data Build Tool) pour créer un schéma en étoile, en utilisant Redshift comme entrepôt de données.

### **Initialisation du projet DBT**

- Installation de la Bibliothèque DBT

    ```
    pip install dbt-redshift
    ```

- Initialisation du projet

    ```
    dbt init
    ```

### **Structurer le projet**

Le projet est organisé autour de trois dossiers principaux sous le répertoire **models**, chacun ayant un rôle spécifique :

- **staging** : Contient les modèles représentant les données brutes de Redshift. Chaque table brute est transformée dans son propre modèle.

- **intermediate** : Contient les modèles intermédiaires utilisés pour préparer les données avant leur intégration dans les tables finales.

- **data_mart** : Contient les modèles pour générer les tables de dimensions et la table de faits finale, utilisées pour l'analyse.

##### **Structure:**
```plaintext
models/
  ├── staging/
  │   ├── stg_customers.sql
  │   ├── stg_items.sql
  │   ├── stg_orders.sql
  │   ├── stg_products.sql
  │   ├── stg_stores.sql
  │   └── stg_supplies.sql
  ├── intermediate/
  │   ├── items_orders_intermediate.sql
  │   ├── customer_orders_intermediate.sql
  │   └── store_orders_intermediate.sql
  └── data_mart/
      ├── dim_customers.sql
      ├── dim_stores.sql
      ├── dim_products_supplies.sql
      ├── dim_date.sql
      └── fact_orders_items.sql
```

#### **Configuration du fichier `sources.yml`**

Le fichier `sources.yml` :

- Définit les sources de données externes (les tables brutes dans Redshift).
- Permet à DBT de référencer les tables dans les modèles, en utilisant des alias clairs et uniformes.
- Facilite la maintenance et les tests grâce à des métadonnées centralisées.



```yaml
version: 2

sources:
  - name: source_redshift   # Nom logique de la source (Redshift brut)
    schema: public          # Schéma de Redshift où se trouvent les tables
    database: dev           # Nom de la base de données Redshift
    tables:
      - name: raw_customers     # Nom de la table brute dans Redshift
        description: "Table contenant les informations sur les clients"
      - name: raw_items
        description: "Table contenant les détails des articles commandés"
      - name: raw_orders
        description: "Table contenant les informations sur les commandes"
      - name: raw_products
        description: "Table contenant les détails des produits disponibles"
      - name: raw_stores
        description: "Table contenant les informations sur les magasins"
      - name: raw_supplies
        description: "Table contenant les informations sur les approvisionnements"
```


#### **Création des vues dans `models/staging/`**

- Les modèles **staging** servent à nettoyer, renommer, et structurer les données brutes provenant de Redshift.
- Ces modèles sont matérialisés sous forme de **vues** pour faciliter les transformations ultérieures.
- Chaque table brute a sa propre vue dans **staging**, ce qui standardise les données et les rend prêtes à être utilisées dans les modèles intermédiaires et finaux.

---

##### **1. `stg_customers.sql`**
```sql
WITH raw_data AS (
    SELECT
        id AS customer_id,
        name AS customer_name
    FROM {{ source('source_redshift', 'raw_customers') }}
)

SELECT *
FROM raw_data
```

---

##### **2. `stg_items.sql`**
```sql
WITH raw_data AS (
    SELECT
        id AS item_id,
        order_id AS order_reference,
        sku AS product_sku
    FROM {{ source('source_redshift', 'raw_items') }}
)

SELECT *
FROM raw_data
```

---

##### **3. `stg_orders.sql`**
```sql
WITH raw_data AS (
    SELECT
        id AS order_id,
        customer_id AS customer_reference,
        ordered_at AS order_date,
        store_id AS store_reference
    FROM {{ source('source_redshift', 'raw_orders') }}
)

SELECT *
FROM raw_data
```

---

##### **4. `stg_products.sql`**
```sql
WITH raw_data AS (
    SELECT
        sku AS product_sku,
        name AS product_name,
        type AS product_type,
        price AS product_price,
        description AS product_description
    FROM {{ source('source_redshift', 'raw_products') }}
)

SELECT *
FROM raw_data
```

---

##### **5. `stg_stores.sql`**
```sql
WITH raw_data AS (
    SELECT
        id AS store_id,
        name AS store_name,
        opened_at AS store_open_date,
        tax_rate AS store_tax_rate
    FROM {{ source('source_redshift', 'raw_stores') }}
)

SELECT *
FROM raw_data
```

---

##### **6. `stg_supplies.sql`**
```sql
WITH raw_data AS (
    SELECT
        id AS supply_id,
        name AS supply_name,
        cost AS supply_cost,
        perishable AS is_perishable,
        sku AS product_sku
    FROM {{ source('source_redshift', 'raw_supplies') }}
)

SELECT *
FROM raw_data
```

---


#### **Création de `dim_date` et utilisation des `dbt packages`**

Dans cette étape, nous utilisons des **`dbt packages`**, qui sont des ensembles de modèles, macros et tests prêts à l'emploi, pour accélérer le développement. Plus précisément, nous importons le package **`dbt_date`** pour créer la table **`dim_date`**, une dimension temporelle essentielle pour les analyses.

##### **Étapes principales :**
1. **Ajout du fichier `packages.yml`** :
   - Ce fichier contient les références aux packages nécessaires, comme `dbt_utils` et `dbt_date`.

2. **Installation des packages** :
   - La commande `dbt deps`  télécharge et intègre les packages dans le projet.

3. **Création de la table `dim_date`** :
   - Le package `dbt_date` est utilisé pour générer automatiquement une table de dimension temporelle avec la macro `get_date_dimension`.

Cette approche simplifie et accélère la génération de **`dim_date`**, en évitant de coder manuellement la logique pour construire une table de dates.

##### **dim_date.sql**
```sql
WITH date_dimension AS (
    {{ dbt_date.get_date_dimension(
        start_date="2016-09-01",
        end_date="2017-08-31"
    ) }}
)

SELECT *
FROM date_dimension

![dim_date](../data/images/dim_date.png)